In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import corner
import astropy.cosmology as cosmo
from astropy.cosmology import Planck18
import astropy.units as unit

seed = 1023123283

# Data Generation

In [2]:
with h5py.File("truth.h5",'r') as hf:
    m = np.array(hf['mtrue'])
    z = np.array(hf['ztrue'])
hf.close()

$$
m_{obs} \sim \mathcal{N}(m_{obs}|m,\sigma_m)
$$

In [3]:
Nobs = m.shape[0]
Nsamp = 4000

In [5]:
sigma_m = 1.0
mo = np.random.normal(m, sigma_m)
ms = np.random.normal(mo[:,None], sigma_m, size=(Nobs,Nsamp))

$$
z_{obs} \sim \mathcal{N}(z_{obs}|z, \sigma_z)
$$

In [7]:
sigma_z = 0.004
zo = np.random.normal(z, sigma_z)
zs = np.random.normal(zo[:,None], sigma_z, size=(Nobs,Nsamp))

In [8]:
print(ms.shape, zs.shape)

(10000, 4000) (10000, 4000)


In [9]:
np.where(ms<0), np.where(zs<0)

((array([], dtype=int64), array([], dtype=int64)),
 (array([], dtype=int64), array([], dtype=int64)))

In [10]:
with h5py.File("pe_samples.h5", "w") as file:    
    file.create_dataset('ms', data=ms)
    file.create_dataset('zs', data=zs)
file.close()

# With Selection

In [11]:
from scipy.interpolate import RectBivariateSpline, interp1d

In [12]:
with h5py.File('optimal_snr.h5', 'r') as inp:
    mass = np.array(inp['ms'])
    osnrs = np.array(inp['SNR'])
inp.close()

osnr_interp = RectBivariateSpline(mass, mass, osnrs)

def optimal_snr(m, z):
    m1z = m*(1.0+z)
    m2z = m*(1.0+z)
    dl = Planck18.luminosity_distance(z).to(unit.Gpc).value
    return osnr_interp.ev(m1z, m2z)/dl

def rho(optimal_snr, Theta):
    return optimal_snr*Theta

In [13]:
Theta = np.random.beta(2.0, 4.0, m.shape[0])

In [14]:
rhoo = rho(optimal_snr(m, z), Theta) + np.random.randn(m.shape[0])

In [15]:
mask = rhoo>8.0
rhoo_det = rhoo[mask]
Ndet = len(rhoo_det)
print(Ndet)

245


In [16]:
ms_det = ms[mask,:]
zs_det = zs[mask,:]

In [17]:
ms_det.shape, zs_det.shape

((245, 4000), (245, 4000))

In [18]:
with h5py.File("pe_samples_w_selection.h5", "w") as file:    
    file.create_dataset('ms_det', data=ms_det)
    file.create_dataset('zs_det', data=zs_det)
file.close()

In [19]:
ms_det.max(), ms_det.min()

(62.1088976154426, 22.919643349590647)

In [20]:
zs_det.max(), zs_det.min()

(1.9772116310228744, 0.003912732753646519)